In [11]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import os
from wettbewerb import load_references, get_3montages
import mne
from scipy import signal as sig
import preprocess
import importlib
import features

In [2]:
training_folder = "../shared_data/training_mini"

In [3]:
ids, channels, data, sampling_frequencies, reference_systems, eeg_labels = load_references(training_folder)

100	 Dateien wurden geladen.


In [4]:
samples_dict = {}
for label, id_, channel, datum, freq, ref in zip(eeg_labels, ids, channels, data, sampling_frequencies, reference_systems):  
    entry = {
        "channel": channel,
        "data": datum,
        "sampling_frequency": freq,
        "reference_system": ref,
        "label": label}
    samples_dict[id_] = entry

In [5]:
# Seperate into seizures and no seizures for feature test reasons 
with_seizure = []
no_seizure = []

for id_, entry in samples_dict.items():
    label = entry["label"]
    if label[0] == 1:
        with_seizure.append(id_)
    else:
        no_seizure.append(id_)

In [6]:
def prefilter(data, fs, notch_freq = 60, Q = 30.0):
    b, a = sig.iirnotch(notch_freq, Q, fs)
    filtered_signal = sig.filtfilt(b,a,data, axis =1)
    return filtered_signal

In [7]:
importlib.reload(features)
%matplotlib inline
example_id = list(samples_dict.keys())[0]
entry = samples_dict[example_id]
signals = entry["data"]  # shape: (n_channels, n_samples)
fs = entry["sampling_frequency"]
ch = entry["channel"] # list len n channels
ref = entry["reference_system"]

preprocessed = preprocess.process_eeg(signals, fs, ch,ref)


      
'''
for id_, entry in samples_dict.items():
    signals = entry["data"]  # shape: (n_channels, n_samples)
    fs = entry["sampling_frequency"]
    
    filtered_signals = prefilter(np.array(signals), fs, notch_freq = 60.0)

    psds = []
    for signal in filtered_signals:
        freqs, psd = sig.welch(signal, fs)
        psds.append(psd)
    
    avg_psd = np.mean(psds, axis=0)

    plt.figure(figsize=(8, 4))
    plt.semilogy(freqs, avg_psd)
    plt.title(f"Avg PSD for ID: {id_} | Seizure: {entry['label'][0] == 1} | Reference System: {entry['reference_system']}")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power Spectral Density (V²/Hz)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
'''

channel types
['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg']
channel_names
['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
signals
(19, 75250)
create info
LE
Creating RawArray with float64 data, n_channels=19, n_times=75250
    Range : 0 ... 75249 =      0.000 ...   300.996 secs
Ready.
pyprep
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 825 samples (3.300 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Executing RANSAC
This may take a while, so be patient...


/home/jupyter-wki_team_3/.conda/envs/wki-sose25/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████|  : 60/60 [00:01<00:00,   44.43it/s]


RANSAC done!
Setting channel interpolation method to {'eeg': 'spline'}.



/home/jupyter-wki_team_3/wki-sose25/preprocess.py:40: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  bads_removed= raw_sig.copy().interpolate_bads()


rereference
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
notch
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 120.00 Hz
-

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


'\nfor id_, entry in samples_dict.items():\n    signals = entry["data"]  # shape: (n_channels, n_samples)\n    fs = entry["sampling_frequency"]\n    \n    filtered_signals = prefilter(np.array(signals), fs, notch_freq = 60.0)\n\n    psds = []\n    for signal in filtered_signals:\n        freqs, psd = sig.welch(signal, fs)\n        psds.append(psd)\n    \n    avg_psd = np.mean(psds, axis=0)\n\n    plt.figure(figsize=(8, 4))\n    plt.semilogy(freqs, avg_psd)\n    plt.title(f"Avg PSD for ID: {id_} | Seizure: {entry[\'label\'][0] == 1} | Reference System: {entry[\'reference_system\']}")\n    plt.xlabel("Frequency (Hz)")\n    plt.ylabel("Power Spectral Density (V²/Hz)")\n    plt.grid(True)\n    plt.tight_layout()\n    plt.show()\n'

In [9]:
importlib.reload(features)

print(features.feature_extraction(preprocessed,fs).shape)


(19, 12)
